In [17]:
import os, sys, warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tqdm

import pandas as pd
import numpy as np
from datetime import datetime as dt
import re

import lightgbm as lgb
from catboost import Pool, CatBoostRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from pygam import LinearGAM, s, f

import optuna
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedShuffleSplit as SSsplit
from sklearn.metrics import mean_squared_error as mse
def rmse(pred, true) : return np.sqrt(mse(true, pred))

import matplotlib.pyplot as plt
import seaborn as sns

## data Load

In [2]:
read_path = '../dataset/fin/'
write_path = '../dataset/result/'
train = pd.read_csv(read_path+'train_fe.csv')
test = pd.read_csv(read_path+'test_fe.csv')

In [3]:
files = os.listdir(read_path)
newFiles = []
histFiles = []
transFiles = []
isFile = re.compile(r'.*[.]csv')
isNew = re.compile(r'.*_new[.]csv')
isHist = re.compile(r'.*_hist[.]csv')
isTrans = re.compile(r'.*_trans[.]csv')
for file in tqdm(files):
    if re.match(isFile, file):
        locals()[file[:-4]] = pd.read_csv(read_path+file)
        if re.match(isNew, file): newFiles.append(file[:-4])
        elif re.match(isHist, file): histFiles.append(file[:-4])
        elif re.match(isTrans, file): transFiles.append(file[:-4])
        else : print('is it proper file name? : {}'.format(file))

is it proper file name? : test_fe.csv
is it proper file name? : train_fe.csv



In [4]:
tempCols = ['merchant_try_'+col if col!='card_id' else col for col in locals()['mertry_trans'].columns.tolist()]
locals()['mertry_trans'].columns = tempCols
tempCols = ['merchant_visit_'+col if col!='card_id' else col for col in locals()['mervisit_trans'].columns.tolist()]
locals()['mervisit_trans'].columns = tempCols

In [5]:
train_trans = train
for file in tqdm(transFiles):
    train_trans = train_trans.merge(locals()[file], how='left', on='card_id')
    if train_trans.shape[0] != train.shape[0] : print('it is wrong : {} : {}'.format(train_trans.shape, file))

In [46]:
modelCols = pd.read_csv('../dataset/modelCols/trans_401.csv')['modelCols'].values.tolist()

catCols = []
isFeature = re.compile(r'feature_[\d]')
isModeKey = re.compile(r'.*_modeKey')

for col in modelCols:
    if re.match(isFeature, col): catCols.append(col)
    elif re.match(isModeKey, col): catCols.append(col)

catCols2 = []
for col in catCols:
    if train_trans[col].isna().sum()==0:
        catCols2.append(modelCols.index(col))

## CV Data

In [48]:
split_rate = test.shape[0]/(train.shape[0]+test.shape[0])
split_y = train['outliers']
SSspliter = SSsplit(3, split_rate)
for i, (train_index, test_index) in enumerate(SSspliter.split(train, split_y)):
    locals()['x_train_'+str(i)] = train_trans[modelCols].iloc[train_index]
    locals()['x_validate_'+str(i)] = train_trans[modelCols].iloc[test_index]
    locals()['y_train_'+str(i)] = train_trans['target'].iloc[train_index]
    locals()['y_validate_'+str(i)] = train_trans['target'].iloc[test_index]

## CV

### LGB

In [38]:
def lgbobj(trial):
    global k
    params = {
    'num_leaves':trial.suggest_int('num_leaves', 10, 1000),
    'max_depth':trial.suggest_int('max_depth', -1, 400),
    'learning_rate':trial.suggest_uniform('learning_rate', 1e-4, 0.1),
    'num_estimators': trial.suggest_int('num_estimators', 500, 1500),
    'subsample_for_bin' : trial.suggest_int('subsample_for_bin', 20, 20000),
    'min_split_gain': trial.suggest_uniform('min_split_gain', 0.,1e-3),
    'min_child_samples': trial.suggest_int('min_child_samples', 1, 200),
    'reg_lambda':trial.suggest_uniform('reg_labmda', 0.1, 0.5),
    'drop_rate': trial.suggest_uniform('drop_rate', 0.01, 0.5),
    'boosting': trial.suggest_categorical('boosting', ['gbdt', 'goss', 'dart']),
    'objective': 'regression',
    'num_threads': 7,
    }
    total_score = 0
    for i in tqdm(range(3)):
        lgb_data = lgb.Dataset(globals()['x_train_'+str(i)], label = globals()['y_train_'+str(i)].values)
        bst = lgb.train(params, lgb_data)
        pred = bst.predict(globals()['x_validate_'+str(i)])
        total_score += rmse(pred, globals()['y_validate_'+str(i)].values)
    print('{} : complete'.format(k))
    k+=1
    return total_score/3
lgbstudy = optuna.create_study()

In [39]:
%%time
k = 0
lgbstudy.optimize(lgbobj, n_trials=300)

0 : complete


[I 2019-02-25 00:16:32,871] Finished a trial resulted in value: 148618257.20726156. Current best value is 148618257.20726156 with parameters: {'num_leaves': 822, 'max_depth': 122, 'learning_rate': 0.011369082947663022, 'num_estimators': 1447, 'subsample_for_bin': 18914, 'min_split_gain': 1.2725743807421753e-05, 'min_child_samples': 163, 'reg_labmda': 0.43877139074078175, 'drop_rate': 0.3020308438775831, 'boosting': 'goss'}.


1 : complete


[I 2019-02-25 00:19:33,656] Finished a trial resulted in value: 3.731566529805566. Current best value is 3.731566529805566 with parameters: {'num_leaves': 835, 'max_depth': 162, 'learning_rate': 0.053659117486951804, 'num_estimators': 697, 'subsample_for_bin': 18684, 'min_split_gain': 0.0007133963113271788, 'min_child_samples': 8, 'reg_labmda': 0.28479801983449626, 'drop_rate': 0.09136349699403186, 'boosting': 'dart'}.


2 : complete


...

52 : complete


[I 2019-02-25 01:48:35,521] Finished a trial resulted in value: 3.7113820770275034. Current best value is 3.685617191493774 with parameters: {'num_leaves': 935, 'max_depth': 245, 'learning_rate': 0.040630726310134826, 'num_estimators': 1436, 'subsample_for_bin': 6342, 'min_split_gain': 0.0004120715698725839, 'min_child_samples': 200, 'reg_labmda': 0.30042871016116973, 'drop_rate': 0.3708866605608058, 'boosting': 'gbdt'}.


53 : complete


[I 2019-02-25 01:49:33,632] Finished a trial resulted in value: 1.3580369511652082e+27. Current best value is 3.685617191493774 with parameters: {'num_leaves': 935, 'max_depth': 245, 'learning_rate': 0.040630726310134826, 'num_estimators': 1436, 'subsample_for_bin': 6342, 'min_split_gain': 0.0004120715698725839, 'min_child_samples': 200, 'reg_labmda': 0.30042871016116973, 'drop_rate': 0.3708866605608058, 'boosting': 'gbdt'}.


KeyboardInterrupt: 

## xgb

In [40]:
def xgb_optuna(trial):
    global k
    model = XGBRegressor(max_depth=trial.suggest_int('max_depth', -1, 400), 
              learning_rate=trial.suggest_uniform('learning_rate', 1e-2, 0.5), 
              n_estimators=trial.suggest_int('n_estimators', 100, 1500),
              booster=trial.suggest_categorical('booster', ['gbtree', 'dart']), 
              min_child_weight=trial.suggest_int('min_child_weight', 1, 200),  
              subsample=trial.suggest_uniform('subsample', 0, 1), 
              reg_alpha=0, 
              reg_lambda=1, 
              scale_pos_weight=1, 
              base_score=0.5,  
              importance_type='gain',
              n_jobs=8,  
              silent=True, 
              objective='reg:linear', 
              gamma=0, 
              max_delta_step=0,
              colsample_bytree=1, 
              colsample_bylevel=1, )
    total_score = 0
    for i in tqdm(range(3)):
        model.fit(globals()['x_train_'+str(i)], globals()['y_train_'+str(i)].values)
        pred = model.predict(globals()['x_validate_'+str(i)])
        total_score += rmse(pred, globals()['y_validate_'+str(i)].values)
    print('{} : complete'.format(k))
    k+=1
    return total_score/3
xgbstudy = optuna.create_study()

In [41]:
%%time
k = 0
xgbstudy.optimize(xgb_optuna, n_trials=100)

Exception ignored in: <function Booster.__del__ at 0x00000201F5728D90>
Traceback (most recent call last):
  File "C:\anaconda3\envs\py37\lib\site-packages\lightgbm\basic.py", line 1554, in __del__
    _safe_call(_LIB.LGBM_BoosterFree(self.handle))
KeyboardInterrupt: 


KeyboardInterrupt: 

## Cat Boost

In [53]:
def cat_optuna(trial):
    global k
    model = CatBoostRegressor(iterations=trial.suggest_int('iterations', 100, 2000),
                        learning_rate=trial.suggest_uniform('learning_rate', 1e-2, 0.5),
#                         depth=trial.suggest_int('depth', 3, 400),
                        silent=True,
                        early_stopping_rounds=10,
                        loss_function='RMSE',
                        thread_count=8,)
    total_score = 0
    for i in tqdm(range(3)):
        train_pool = Pool(globals()['x_train_'+str(i)], globals()['y_train_'+str(i)].values, cat_features=catCols2)
        test_pool = Pool(globals()['x_validate_'+str(i)], cat_features=catCols2) 
        model.fit(train_pool, silent=True)
        pred = model.predict(test_pool)
        total_score += rmse(pred, globals()['y_validate_'+str(i)].values)
    print('{} : complete'.format(k))
    k+=1
    return total_score/3
catstudy = optuna.create_study()

In [54]:
%%time
k = 0
catstudy.optimize(cat_optuna, n_trials=100)

0 : complete


[I 2019-02-25 02:49:37,870] Finished a trial resulted in value: 3.7228686208535096. Current best value is 3.7228686208535096 with parameters: {'iterations': 840, 'learning_rate': 0.1893025064860477}.


1 : complete


[I 2019-02-25 02:53:42,722] Finished a trial resulted in value: 3.7218307265627417. Current best value is 3.7218307265627417 with parameters: {'iterations': 523, 'learning_rate': 0.22775658852973543}.


2 : complete


...

15 : complete


[I 2019-02-25 04:49:08,769] Finished a trial resulted in value: 3.8665922224123546. Current best value is 3.689683078775776 with parameters: {'iterations': 878, 'learning_rate': 0.042356140476210384}.


16 : complete


[I 2019-02-25 04:56:43,347] Finished a trial resulted in value: 3.710625775842889. Current best value is 3.689683078775776 with parameters: {'iterations': 878, 'learning_rate': 0.042356140476210384}.


KeyboardInterrupt: 